# Zack Larsen CSC 478 Assignment 4
Due: Thursday, November 17

For this assignment you will experiment with Principal Component Analysis as a dimensionality reduction approach to assist in clustering high-dimensional data. You will also perform association rule mining using the implementation provided in the textbook (Machine Learning in Action, Chapter 11). Finally, you'll experiment with item-based recommendation for a joke recommender system.



# 1
PCA for Reduced Dimensionality in Clustering [Dataset: segmentation_data.zip]
For this problem you will use an image segmentation data set for clustering. You will experiment with using PCA as an approach to reduce dimensionality and noise in the data. You will compare the results of clustering the data with and without PCA using the provided image class assignments as the ground truth. The data set is divided into three files. 

The file "segmentation_data.txt" contains data about images with each line corresponding to one image. Each image is represented by 19 features (these are the columns in the data and correspond to the feature names in the file "segmentation_names.txt". 

The file "segmentation_classes.txt" contains the class labels (the type of image) and a numeric class label for each of the corresponding images in the data file. After clustering the image data, you will use the class labels to measure completeness and homogeneity of the generated clusters. The data set used in this problem is based on the Image Segmentation data set at the UCI Machine Learning Repository.
Your tasks in this problem are the following:

In [1]:
cd /Users/zacklarsen/Desktop/CSC 478/HW/HW #4

/Users/zacklarsen/Desktop/CSC 478/HW/HW #4


In [2]:
import apriori2 as ap

In [3]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from sklearn.cross_validation import KFold
import pylab as pl
from sklearn import preprocessing
from sklearn.cluster import KMeans

# 1a
Load in the image data matrix (with rows as images and columns as features). Also load in the numeric class labels from the segmentation class file. Using your favorite method (e.g., sklearn's min-max scaler), perform min-max normalization on the data matrix so that each feature is scaled to [0,1] range.

In [4]:
image = pd.read_csv('segmentation_data.txt',na_values=["?"],header=None)
image.shape

(2100, 19)

In [21]:
np.set_printoptions(precision=5,suppress=True)
image

array([[ 0.43083,  0.74167,  0.     , ...,  0.12371,  0.50814,  0.83185],
       [ 0.33597,  0.73333,  0.     , ...,  0.12739,  0.46333,  0.83699],
       [ 0.88538,  0.97083,  0.     , ...,  0.1134 ,  0.48015,  0.84478],
       ..., 
       [ 0.50198,  0.625  ,  0.     , ...,  0.07216,  0.54092,  0.17592],
       [ 0.58893,  0.6125 ,  0.     , ...,  0.081  ,  0.50309,  0.18479],
       [ 0.48617,  0.62917,  0.     , ...,  0.09647,  0.47993,  0.17037]])

In [6]:
classes = pd.read_csv('segmentation_classes.txt',na_values=["?"],header=None,sep="\t")
classes.shape

(2100, 2)

In [50]:
print classes.head()

       0  1
0  GRASS  0
1  GRASS  0
2  GRASS  0
3  GRASS  0
4  GRASS  0


In [53]:
class_names = pd.read_csv('segmentation_names.txt',header=None)

In [74]:
class_names[0][1]

'REGION-CENTROID-ROW'

In [29]:
target = classes[1]

In [30]:
target.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: 1, dtype: int64

min-max scaling using sklearn

In [31]:
min_max_scaler = preprocessing.MinMaxScaler()
image = min_max_scaler.fit_transform(image)

In [32]:
np.set_printoptions(precision=5,suppress=True)
image   #Look at image to verify the scale is between 0 and 1

array([[ 0.43083,  0.74167,  0.     , ...,  0.12371,  0.50814,  0.83185],
       [ 0.33597,  0.73333,  0.     , ...,  0.12739,  0.46333,  0.83699],
       [ 0.88538,  0.97083,  0.     , ...,  0.1134 ,  0.48015,  0.84478],
       ..., 
       [ 0.50198,  0.625  ,  0.     , ...,  0.07216,  0.54092,  0.17592],
       [ 0.58893,  0.6125 ,  0.     , ...,  0.081  ,  0.50309,  0.18479],
       [ 0.48617,  0.62917,  0.     , ...,  0.09647,  0.47993,  0.17037]])

# 1b 
Next, Perform Kmeans clustering (for this problem, use the Kmeans implementation in scikit-learn) on the image data (since there are a total 7 pre-assigned image classes, you should use K = 7 in your clustering).

Use Euclidean distance as your distance measure for the clustering. 

In [33]:
kmeans = KMeans(n_clusters=7, max_iter=500, verbose=1) # initialization

In [34]:
kmeans.fit(image)

Initialization complete
Iteration  0, inertia 492.348
Iteration  1, inertia 384.454
Iteration  2, inertia 377.565
Iteration  3, inertia 374.048
Iteration  4, inertia 371.606
Iteration  5, inertia 370.071
Iteration  6, inertia 369.928
Iteration  7, inertia 369.888
Iteration  8, inertia 369.866
Iteration  9, inertia 369.861
Iteration 10, inertia 369.859
Converged at iteration 10
Initialization complete
Iteration  0, inertia 641.321
Iteration  1, inertia 395.135
Iteration  2, inertia 370.378
Iteration  3, inertia 355.448
Iteration  4, inertia 350.600
Iteration  5, inertia 350.156
Iteration  6, inertia 350.057
Iteration  7, inertia 350.018
Iteration  8, inertia 350.013
Converged at iteration 8
Initialization complete
Iteration  0, inertia 554.114
Iteration  1, inertia 389.132
Iteration  2, inertia 373.200
Iteration  3, inertia 369.545
Iteration  4, inertia 368.195
Iteration  5, inertia 367.255
Iteration  6, inertia 366.929
Iteration  7, inertia 366.537
Iteration  8, inertia 366.226
Iterati

KMeans(copy_x=True, init='k-means++', max_iter=500, n_clusters=7, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=1)

Print the cluster centroids (use some formatting so that they are visually understandable). 

In [35]:
clusters = kmeans.predict(image)

In [36]:
centroids = kmeans.cluster_centers_

In [43]:
np.set_printoptions(precision=5,suppress=True)
print centroids

[[ 0.5351   0.15017  0.       0.02778  0.00167  0.03023  0.00054  0.02677
   0.00059  0.82325  0.77972  0.89417  0.78876  0.27067  0.66637  0.28939
   0.89417  0.2118   0.12507]
 [ 0.77067  0.42522  0.       0.01398  0.02258  0.04024  0.00299  0.02312
   0.00209  0.04114  0.03501  0.05834  0.02877  0.77847  0.22394  0.48651
   0.05929  0.53818  0.24397]
 [ 0.7507   0.53456  0.       0.04     0.03846  0.11442  0.0193   0.10924
   0.018    0.30096  0.27991  0.35274  0.26592  0.59257  0.45112  0.30916
   0.35282  0.30234  0.16476]
 [ 0.30251  0.53086  0.       0.05226  0.04661  0.10082  0.00942  0.08397
   0.01104  0.40061  0.37035  0.47246  0.35304  0.49715  0.57088  0.21305
   0.47246  0.30226  0.16388]
 [ 0.51399  0.80894  0.       0.07744  0.00505  0.05447  0.00141  0.04633
   0.0014   0.10879  0.0914   0.09241  0.14268  0.67916  0.079    0.82129
   0.1349   0.41449  0.89233]
 [ 0.25417  0.45997  0.       0.02626  0.0137   0.03727  0.00236  0.02787
   0.00202  0.02601  0.01793  0.0423

In [45]:
print len(clusters)

2100


In [89]:
print centroids[0]

[ 0.5351   0.15017  0.       0.02778  0.00167  0.03023  0.00054  0.02677
  0.00059  0.82325  0.77972  0.89417  0.78876  0.27067  0.66637  0.28939
  0.89417  0.2118   0.12507]


In [151]:
#print "Centroid 0\tCentroid 1\tCentroid 2\tCentroid 3\tCentroid 4\tCentroid 5\tCentroid 6"

#for i in range(len(class_names)):
 #   print "%.8f\t%.8f\t%.8f\t%.8f\t%.8f\t%.8f\t%.8f" %(centroids[0][i],centroids[1][i],centroids[2][i],centroids[3][i],centroids[4][i],centroids[5][i],centroids[6][i])  

In [152]:
print "Class Name\t\tCluster 0\tCluster 1\tCluster 2\tCluster 3\tCluster 4\tCluster 5"
for i in range(len(class_names)):
    print "%20s\t%.8f\t%.8f\t%.8f\t%.8f\t%.8f\t%.8f" %(class_names[0][i],centroids[0][i],centroids[1][i],centroids[2][i],centroids[3][i],centroids[4][i],centroids[5][i])


print "And also cluster 6: "

print "Class Name\t\tCluster 6"
for i in range(len(class_names)):
    print "%20s\t%.8f" %(class_names[0][i],centroids[6][i])

Class Name		Centroid 0	Centroid 1	Centroid 2	Centroid 3	Centroid 4	Centroid 5
 REGION-CENTROID-COL	0.53509881	0.77067449	0.75069626	0.30250553	0.51399369	0.25416915
 REGION-CENTROID-ROW	0.15016667	0.42521505	0.53456410	0.53086158	0.80893659	0.45997432
  REGION-PIXEL-COUNT	0.00000000	0.00000000	0.00000000	0.00000000	0.00000000	0.00000000
SHORT-LINE-DENSITY-5	0.02777778	0.01397849	0.04000000	0.05225989	0.07744108	0.02625571
SHORT-LINE-DENSITY-2	0.00166667	0.02258065	0.03846154	0.04661017	0.00505051	0.01369863
          VEDGE-MEAN	0.03022814	0.04023672	0.11441942	0.10081685	0.05447376	0.03727408
            VEDGE-SD	0.00054289	0.00298876	0.01930058	0.00942022	0.00140719	0.00236373
          HEDGE-MEAN	0.02676605	0.02312162	0.10924032	0.08397199	0.04633498	0.02787373
            HEDGE-SD	0.00058666	0.00208840	0.01799891	0.01104328	0.00140097	0.00201698
      INTENSITY-MEAN	0.82324643	0.04113840	0.30095534	0.40060809	0.10878994	0.02601254
         RAWRED-MEAN	0.77971638	0.03500549	0.2799077

Compare your 7 clusters to the 7 pre-assigned classes by computing the Completeness and Homogeneity values of the generated clusters.

In [153]:
from sklearn.metrics import completeness_score, homogeneity_score

In [154]:
print completeness_score(target,clusters)

0.613187012485


In [155]:
print homogeneity_score(target,clusters)

0.611502116337


# 1c
Perform PCA on the normalized image data matrix. You may use the linear algebra package in Numpy or the Decomposition module in scikit-learn (the latter is much more efficient). 

Analyze the principal components to determine the number, r, of PCs needed to capture at least 95% of variance in the data. 

Then use these r components as features to transform the data into a reduced dimension space. [See the PCA Clustering Notebook from class for an example of how these steps are performed.]

In [157]:
from sklearn.decomposition import PCA as PCA

In [158]:
pca = PCA(n_components=10)
image_trans = pca.fit(image).transform(image)

In [162]:
np.set_printoptions(precision=4,suppress=True)
image_trans

array([[-0.6891,  0.533 , -0.2461, ...,  0.0049, -0.    , -0.0036],
       [-0.6669,  0.5107, -0.338 , ...,  0.0195, -0.0007, -0.0056],
       [-0.712 ,  0.7709,  0.1558, ..., -0.0495,  0.0479,  0.0441],
       ..., 
       [-0.5077, -0.1294,  0.0822, ..., -0.0257,  0.0084, -0.0759],
       [-0.4791, -0.0863,  0.1587, ..., -0.0345,  0.033 , -0.0496],
       [-0.4423, -0.1062,  0.0471, ..., -0.1252,  0.0463, -0.1071]])

In [163]:
np.set_printoptions(precision=8,suppress=True)

print(pca.explained_variance_ratio_)

[ 0.60714234  0.13196979  0.10123773  0.04543539  0.03547361  0.01988035
  0.0189197   0.01615409  0.01065601  0.00711338]


In [180]:
num_PC = 6
cumulative_variance = 0
for i in range(num_PC):
    cumulative_variance += pca.explained_variance_ratio_[i]
    
print "Cumulative variance with",num_PC,"components is: ",cumulative_variance

Cumulative variance with 6 components is:  0.941139219796


In [179]:
num_PC = 7
cumulative_variance = 0
for i in range(num_PC):
    cumulative_variance += pca.explained_variance_ratio_[i]
    
print "Cumulative variance with",num_PC,"components is: ",cumulative_variance

Cumulative variance with 7 components is:  0.96005892277


Looking at the explained variance ratio from PCA above, we need the first 7 PC's to account for at least 95% of the variance in the data.

We can reduce the dimensionality of the data below:

In [181]:
pca = PCA(n_components=7)
image_trans = pca.fit(image).transform(image)

# 1d
Perform Kmeans again, but this time on the lower dimensional transformed data. Then, compute the Completeness and Homogeneity values of the new clusters.

In [182]:
kmeans = KMeans(n_clusters=7, max_iter=500, verbose=1) # initialization

In [183]:
kmeans.fit(image_trans)

Initialization complete
Iteration  0, inertia 580.546
Iteration  1, inertia 340.628
Iteration  2, inertia 319.993
Iteration  3, inertia 312.652
Iteration  4, inertia 308.425
Iteration  5, inertia 306.956
Iteration  6, inertia 306.645
Iteration  7, inertia 306.562
Iteration  8, inertia 306.500
Iteration  9, inertia 306.434
Iteration 10, inertia 306.409
Iteration 11, inertia 306.388
Iteration 12, inertia 306.374
Iteration 13, inertia 306.367
Converged at iteration 13
Initialization complete
Iteration  0, inertia 407.197
Iteration  1, inertia 316.673
Iteration  2, inertia 310.518
Iteration  3, inertia 308.130
Iteration  4, inertia 307.029
Iteration  5, inertia 306.622
Iteration  6, inertia 306.248
Iteration  7, inertia 306.109
Iteration  8, inertia 306.022
Iteration  9, inertia 306.005
Iteration 10, inertia 305.987
Iteration 11, inertia 305.976
Iteration 12, inertia 305.971
Converged at iteration 12
Initialization complete
Iteration  0, inertia 442.918
Iteration  1, inertia 303.571
Iterat

KMeans(copy_x=True, init='k-means++', max_iter=500, n_clusters=7, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=1)

In [184]:
clusters = kmeans.predict(image_trans)

In [185]:
centroids = kmeans.cluster_centers_

In [186]:
from sklearn.metrics import completeness_score, homogeneity_score

In [187]:
print completeness_score(target,clusters)

0.611812149028


In [188]:
print homogeneity_score(target,clusters)

0.610164346851


# 1e
Discuss your observations based on the comparison of the two clustering results.

# 1e discussion
The completeness score from the kmeans before PCA was 0.613187012485.
The homogeneity score from the kmeans before PCA was 0.611502116337.

The completeness score from the kmeans after PCA was 0.611812149028
The homogeneity score from the kmeans after PCA was 0.610164346851.

We see from the results that we obtain lower scores for both completeness and homogeneity after performing PCA on the data. These results are fairly close to the results before PCA, so this might lead us to conclude that we can safely perform PCA for dimensionality reduction without suffering a severe decrease in our model's clustering performance.

# Question 2
Association Rule Discovery [Dataset: playlists.zip]
For this problem you will experiment with association rule mining using Apriori algorithm discussed in class. You will use a modified version of the Apriori implementation in Machine Learning in Action (it has been modified to compute lift values for rules in addition to confidence). [See Associations Notebook from class for an example of using this module.]

The data set you will use is based on a music playlist data set obtained from Yes.com. [See the full description of this data]. We will only use a portion of this data. The provided data archive contains two files. 

The file "playlists.txt" contains on each line a sequence of songs played as part of one playlist. The songs are represented by integer values. 

The file "song_names.txt" contains the mapping between the integer codes and song tiles and artists (format of the song names is [song title]::[artist]). You will need both of these files to generate association rules.
Your tasks in this problem are the following:

# 2a
Load the playlist data into a Python nested list, and the song_names data into an appropriate data structure.

As discussed in class, the order of the playlists occurring does not matter

In [189]:
#This is the code from the class example notebook

playlist = [line.strip().split(',') for line in open('playlists.txt').readlines()]

In [190]:
playlist

[['0 1 2 3 4 5 6 7 8'],
 ['9 10 11'],
 ['12 13 14 15'],
 ['16 17 18'],
 ['19'],
 ['2'],
 ['20 21 22 15 23 8 24 25'],
 ['26'],
 ['27 18'],
 ['28 29'],
 ['30 3 31'],
 ['20 21'],
 ['2 22 32 33 34 27 24 35'],
 ['26'],
 ['18 36 28 15 20'],
 ['37 28 15'],
 ['102'],
 ['172'],
 ['173 53'],
 ['174'],
 ['175'],
 ['176'],
 ['199'],
 ['200'],
 ['201'],
 ['212'],
 ['215'],
 ['314 211 344'],
 ['230 401'],
 ['257 323 286 224 461 416 274'],
 ['242'],
 ['222 334 288 230'],
 ['295 262 219 229 215'],
 ['209 207 242 434 462 220'],
 ['405'],
 ['311'],
 ['224 400 212 243 218 359'],
 ['209 213 239 264 300 237 415'],
 ['367'],
 ['222 292 463 262 218 357'],
 ['212 351 293'],
 ['238 395 209 455 232 280 254 397 257 327'],
 ['249 464 209 277 458 353 262 341 246 283 211 360 355 285 241 465'],
 ['222 352 376'],
 ['237 311 348 250 212'],
 ['290'],
 ['206'],
 ['245 300 304 230 411 445 320 242 210 331 265 204 466 257'],
 ['223 306 274 253'],
 ['467 372 212 275 409 377 218 307 242 337 376 236 324 222'],
 ['416'],
 ['45

In [191]:
sessions = []

for s in playlist:
    t = [u.rstrip() for u in s]
    #print t[0]
    sessions.append((t[0].split()))

In [192]:
sessions

[['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 ['9', '10', '11'],
 ['12', '13', '14', '15'],
 ['16', '17', '18'],
 ['19'],
 ['2'],
 ['20', '21', '22', '15', '23', '8', '24', '25'],
 ['26'],
 ['27', '18'],
 ['28', '29'],
 ['30', '3', '31'],
 ['20', '21'],
 ['2', '22', '32', '33', '34', '27', '24', '35'],
 ['26'],
 ['18', '36', '28', '15', '20'],
 ['37', '28', '15'],
 ['102'],
 ['172'],
 ['173', '53'],
 ['174'],
 ['175'],
 ['176'],
 ['199'],
 ['200'],
 ['201'],
 ['212'],
 ['215'],
 ['314', '211', '344'],
 ['230', '401'],
 ['257', '323', '286', '224', '461', '416', '274'],
 ['242'],
 ['222', '334', '288', '230'],
 ['295', '262', '219', '229', '215'],
 ['209', '207', '242', '434', '462', '220'],
 ['405'],
 ['311'],
 ['224', '400', '212', '243', '218', '359'],
 ['209', '213', '239', '264', '300', '237', '415'],
 ['367'],
 ['222', '292', '463', '262', '218', '357'],
 ['212', '351', '293'],
 ['238', '395', '209', '455', '232', '280', '254', '397', '257', '327'],
 ['249',
  '464',
  '209',
 

In [193]:
for i in range(len(sessions)):
    sessions[i][:] = [int(x) for x in sessions[i]]

In [194]:
sessions

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18],
 [19],
 [2],
 [20, 21, 22, 15, 23, 8, 24, 25],
 [26],
 [27, 18],
 [28, 29],
 [30, 3, 31],
 [20, 21],
 [2, 22, 32, 33, 34, 27, 24, 35],
 [26],
 [18, 36, 28, 15, 20],
 [37, 28, 15],
 [102],
 [172],
 [173, 53],
 [174],
 [175],
 [176],
 [199],
 [200],
 [201],
 [212],
 [215],
 [314, 211, 344],
 [230, 401],
 [257, 323, 286, 224, 461, 416, 274],
 [242],
 [222, 334, 288, 230],
 [295, 262, 219, 229, 215],
 [209, 207, 242, 434, 462, 220],
 [405],
 [311],
 [224, 400, 212, 243, 218, 359],
 [209, 213, 239, 264, 300, 237, 415],
 [367],
 [222, 292, 463, 262, 218, 357],
 [212, 351, 293],
 [238, 395, 209, 455, 232, 280, 254, 397, 257, 327],
 [249,
  464,
  209,
  277,
  458,
  353,
  262,
  341,
  246,
  283,
  211,
  360,
  355,
  285,
  241,
  465],
 [222, 352, 376],
 [237, 311, 348, 250, 212],
 [290],
 [206],
 [245, 300, 304, 230, 411, 445, 320, 242, 210, 331, 265, 204, 466, 257],
 [223, 306, 274, 253],
 [467, 372, 212, 27

Import song_names data into appropriate data structure

In [195]:
song_names = open('song_names.txt','r')

In [196]:
song_lines = song_names.readlines()

In [197]:
song_names.close()

In [198]:
song_list = []
for line in song_lines:
    items = line.rstrip('\r\n').split('\t')   # strip new-line characters and split on column delimiter
    items = [item.strip() for item in items]  # strip extra whitespace off data items
    song_list.append(items)

In [199]:
song_list

[['0', 'Gucci-Time::Gucci-Mane'],
 ['1', 'Aston-Martin-Music::Rick-Ross'],
 ['2', 'Get-Back-Up::T.I.'],
 ['3', 'Hot-Toddy::Usher'],
 ['4', 'Whip-My-Hair::Willow'],
 ['5', 'Down-On-Me::Jeremih'],
 ['6', 'Black-And-Yellow::Wiz-Khalifa'],
 ['7', 'Blowing-Me-Kisses::Soulja-Boy'],
 ['8', 'Lay-It-Down::Lloyd'],
 ['9', "Who's-That-Chick::Rihanna"],
 ['10', 'Like-A-G6::Far-East-Movement'],
 ['11', "DJ-Got-Us-Fallin'-In-Love::Usher"],
 ['12', 'All-I-Wants-Is-You::Miguel'],
 ['13', 'Champagne-Life::Ne-Yo'],
 ['14', 'Find-Your-Love::Drake'],
 ['15', 'Your-Love::Nicki-Minaj'],
 ['16', 'Ms.-Jackson::Outkast'],
 ['17', 'Close-To-You::Maxi-Priest'],
 ['18', 'Right-Thru-Me::Nicki-Minaj'],
 ['19', 'You-Make-Me-Wanna...::Usher'],
 ['20', "What's-My-Name::Rihanna"],
 ['21', 'One-In-A-Million::Ne-Yo'],
 ['22', 'Check-It-Out::Will.I.Am'],
 ['23', 'Be-Without-You::Mary-J.-Blige'],
 ['24', 'Bottoms-Up::Trey-Songz'],
 ['25', 'There-Goes-My-Baby::Usher'],
 ['26', 'Up-All-Night::Drake'],
 ['27', 'Liquor-Store-B

In [200]:
pairs = []
for elem in song_list:
    pairs.append(elem[1])

print pairs[1]

Aston-Martin-Music::Rick-Ross


In [201]:
artist_song_dictionary = {}

for i in range(len(pairs)):
    pairstrip = pairs[i].strip()
    pairsplit = pairstrip.split("::")
    artist_song_dictionary[pairsplit[1]] = pairsplit[0]
    

In [202]:
artist_song_dictionary

{'Celine-Dion': "That's-The-Way-It-Is",
 'Young-Jeezy': 'Lose-My-Mind',
 'Palomo': 'Aun-Suspiro',
 'The-Romantics': 'What-I-Like-About-You',
 'Brenda-Russell': 'Piano-In-The-Dark',
 'The-Verve': 'Bitter-Sweet-Symphony',
 'Beyonce': 'Irreplaceable',
 'Plumb': 'In-My-Arms',
 'Brad-Paisley': 'Old-Alabama',
 'Powerman-5000': 'When-Worlds-Collide',
 'Kumbia-All-Starz': 'Chiquilla',
 'Thin-Lizzy': 'The-Boys-Are-Back-In-Town',
 'Prince-Royce': 'El-Amor-Que-Perdimos',
 'Jenni-Rivera': 'La-Gran-Senora',
 'Kevin-LeVar-&-One-Sound': 'A-Heart-That-Forgives',
 'Los-Tiranos-Del-Norte': 'Hasta-La-Miel-Amarga',
 'DJ-Khaled': '"Welcome-To-My-Hood"',
 'Warren-Bernhardt': 'Felinicity',
 'Ekolu': 'Your-Lovin-Is-Nice',
 'James-Otto': 'Soldiers-&-Jesus',
 'Seether': 'Country-Song',
 'Norman-Brown': 'Sending-My-Love',
 'Jose-Feliciano': 'Feliz-Navidad',
 'Quiet-Riot': '(Bang-Your-Head)-Metal-Health',
 'Avenged-Sevenfold': 'Welcome-To-The-Family',
 'Heatwave': 'Always-And-Forever',
 'Gocho': 'Dandole',
 'Jaso

In [203]:
artist_song_dictionary['Powerman-5000']

'When-Worlds-Collide'

In [204]:
artist_song_dictionary['Beyonce']

'Irreplaceable'

In [205]:
song_list

[['0', 'Gucci-Time::Gucci-Mane'],
 ['1', 'Aston-Martin-Music::Rick-Ross'],
 ['2', 'Get-Back-Up::T.I.'],
 ['3', 'Hot-Toddy::Usher'],
 ['4', 'Whip-My-Hair::Willow'],
 ['5', 'Down-On-Me::Jeremih'],
 ['6', 'Black-And-Yellow::Wiz-Khalifa'],
 ['7', 'Blowing-Me-Kisses::Soulja-Boy'],
 ['8', 'Lay-It-Down::Lloyd'],
 ['9', "Who's-That-Chick::Rihanna"],
 ['10', 'Like-A-G6::Far-East-Movement'],
 ['11', "DJ-Got-Us-Fallin'-In-Love::Usher"],
 ['12', 'All-I-Wants-Is-You::Miguel'],
 ['13', 'Champagne-Life::Ne-Yo'],
 ['14', 'Find-Your-Love::Drake'],
 ['15', 'Your-Love::Nicki-Minaj'],
 ['16', 'Ms.-Jackson::Outkast'],
 ['17', 'Close-To-You::Maxi-Priest'],
 ['18', 'Right-Thru-Me::Nicki-Minaj'],
 ['19', 'You-Make-Me-Wanna...::Usher'],
 ['20', "What's-My-Name::Rihanna"],
 ['21', 'One-In-A-Million::Ne-Yo'],
 ['22', 'Check-It-Out::Will.I.Am'],
 ['23', 'Be-Without-You::Mary-J.-Blige'],
 ['24', 'Bottoms-Up::Trey-Songz'],
 ['25', 'There-Goes-My-Baby::Usher'],
 ['26', 'Up-All-Night::Drake'],
 ['27', 'Liquor-Store-B

In [206]:
len(song_list)

3168

In [207]:
song_list[1]

['1', 'Aston-Martin-Music::Rick-Ross']

In [208]:
song_list[1][1]

'Aston-Martin-Music::Rick-Ross'

In [209]:
index_song_dictionary = {}

for i in range(len(song_list)):
    #print i
    index_song_dictionary[i] = song_list[i][1]  

In [210]:
index_song_dictionary

{0: 'Gucci-Time::Gucci-Mane',
 1: 'Aston-Martin-Music::Rick-Ross',
 2: 'Get-Back-Up::T.I.',
 3: 'Hot-Toddy::Usher',
 4: 'Whip-My-Hair::Willow',
 5: 'Down-On-Me::Jeremih',
 6: 'Black-And-Yellow::Wiz-Khalifa',
 7: 'Blowing-Me-Kisses::Soulja-Boy',
 8: 'Lay-It-Down::Lloyd',
 9: "Who's-That-Chick::Rihanna",
 10: 'Like-A-G6::Far-East-Movement',
 11: "DJ-Got-Us-Fallin'-In-Love::Usher",
 12: 'All-I-Wants-Is-You::Miguel',
 13: 'Champagne-Life::Ne-Yo',
 14: 'Find-Your-Love::Drake',
 15: 'Your-Love::Nicki-Minaj',
 16: 'Ms.-Jackson::Outkast',
 17: 'Close-To-You::Maxi-Priest',
 18: 'Right-Thru-Me::Nicki-Minaj',
 19: 'You-Make-Me-Wanna...::Usher',
 20: "What's-My-Name::Rihanna",
 21: 'One-In-A-Million::Ne-Yo',
 22: 'Check-It-Out::Will.I.Am',
 23: 'Be-Without-You::Mary-J.-Blige',
 24: 'Bottoms-Up::Trey-Songz',
 25: 'There-Goes-My-Baby::Usher',
 26: 'Up-All-Night::Drake',
 27: 'Liquor-Store-Blues::Bruno-Mars',
 28: '"Monster"::Kanye-West',
 29: 'Just-The-Way-You-Are::Bruno-Mars',
 30: 'Runaway::Kanye-

# 2b
Run Apriori on the playlist data using a min-support value of 0.002.

Don't use the book's implementation for this problem. Use the apriori2.py implementation from the class example notebook.

In [1]:
import numpy as np
import pandas as pd
import apriori2 as ap

ImportError: No module named apriori2

In [217]:
#D = ap.loadDataSet()   #Here, we are using a sample set

In [218]:
#print D

In [219]:
#L,support = ap.apriori(D,0.02)

In [220]:
#L

In [221]:
#support

# Here, we are using the real data set sessions from the listening sessions we loaded in above

In [222]:
L,support = ap.apriori(sessions,0.002)

In [223]:
L

[[frozenset({1938}),
  frozenset({2776}),
  frozenset({2508}),
  frozenset({2792}),
  frozenset({1250}),
  frozenset({2841}),
  frozenset({3019}),
  frozenset({1769}),
  frozenset({195}),
  frozenset({546}),
  frozenset({3046}),
  frozenset({3094}),
  frozenset({2116}),
  frozenset({1448}),
  frozenset({2117}),
  frozenset({2057}),
  frozenset({405}),
  frozenset({490}),
  frozenset({786}),
  frozenset({1616}),
  frozenset({1749}),
  frozenset({2836}),
  frozenset({29}),
  frozenset({2747}),
  frozenset({1057}),
  frozenset({78}),
  frozenset({122}),
  frozenset({175}),
  frozenset({2955}),
  frozenset({1996}),
  frozenset({232}),
  frozenset({2826}),
  frozenset({372}),
  frozenset({3007}),
  frozenset({2966}),
  frozenset({807}),
  frozenset({2267}),
  frozenset({2229}),
  frozenset({2762}),
  frozenset({2680}),
  frozenset({2417}),
  frozenset({2923}),
  frozenset({26}),
  frozenset({196}),
  frozenset({636}),
  frozenset({701}),
  frozenset({3052}),
  frozenset({2998}),
  frozenset

In [224]:
support

{frozenset({241, 330}): 9.643201542912247e-05,
 frozenset({215, 2776}): 0.00012054001928640309,
 frozenset({2609}): 0.0024108003857280617,
 frozenset({511, 565}): 0.0001687560270009643,
 frozenset({1397, 2966}): 4.8216007714561236e-05,
 frozenset({552, 637}): 0.00024108003857280618,
 frozenset({2379}): 0.0005785920925747348,
 frozenset({565, 2917}): 0.00012054001928640309,
 frozenset({1988, 2037}): 0.0002651880424300868,
 frozenset({2790, 2866, 2924}): 0.0011089681774349085,
 frozenset({78, 2572}): 9.643201542912247e-05,
 frozenset({2415, 2940}): 0.00019286403085824494,
 frozenset({520, 2417}): 0.00019286403085824494,
 frozenset({319}): 0.002627772420443587,
 frozenset({1250, 2552}): 2.4108003857280618e-05,
 frozenset({2020, 2047}): 0.0001687560270009643,
 frozenset({285, 2584}): 0.0003857280617164899,
 frozenset({2087, 2417}): 2.4108003857280618e-05,
 frozenset({405, 2812}): 7.232401157184185e-05,
 frozenset({2671}): 0.0005303760848601736,
 frozenset({796}): 0.0009884281581485053,
 fr

# 2c
Generate rules from the frequent itemsets discovered in the previous step (you can try different metrics ('lift' or 'confidence' to see which gives you more useful and interesting results). At minimum, you should generate rules with a minimum lift value of 20.0. If you use confidence, you may want to set the min-confidence threshold to about 0.5.

In [225]:
liftruleList = ap.generateRules(L,support,metric='lift',minMetric=20)

frozenset([34]) --> frozenset([80]) conf: 0.364985163205  lift: 28.4578657326
frozenset([80]) --> frozenset([34]) conf: 0.231203007519  lift: 28.4578657326
frozenset([2791]) --> frozenset([634]) conf: 0.297935103245  lift: 37.9090431981
frozenset([634]) --> frozenset([2791]) conf: 0.30981595092  lift: 37.9090431981
frozenset([3003]) --> frozenset([635]) conf: 0.404255319149  lift: 47.3686741195
frozenset([635]) --> frozenset([3003]) conf: 0.268361581921  lift: 47.3686741195
frozenset([2774]) --> frozenset([20]) conf: 0.440298507463  lift: 27.9259664978
frozenset([20]) --> frozenset([2774]) conf: 0.180428134557  lift: 27.9259664978
frozenset([20]) --> frozenset([18]) conf: 0.227828746177  lift: 27.0009611184
frozenset([18]) --> frozenset([20]) conf: 0.425714285714  lift: 27.0009611184
frozenset([89]) --> frozenset([2840]) conf: 0.308823529412  lift: 27.3134328358
frozenset([2840]) --> frozenset([89]) conf: 0.268656716418  lift: 27.3134328358
frozenset([20]) --> frozenset([34]) conf: 0.1

In [171]:
#print liftruleList

In [226]:
confruleList = ap.generateRules(L,support,metric='confidence',minMetric=0.5)

frozenset([2840]) --> frozenset([2839]) conf: 0.511727078891  lift: 29.1571967478
frozenset([2883]) --> frozenset([2878]) conf: 0.528138528139  lift: 49.9024741394
frozenset([2826]) --> frozenset([2839]) conf: 0.617511520737  lift: 35.1845850003
frozenset([2774]) --> frozenset([2018]) conf: 0.514925373134  lift: 34.843563585
frozenset([2878]) --> frozenset([2839]) conf: 0.507972665148  lift: 28.943277679
frozenset([2878]) --> frozenset([5]) conf: 0.535307517084  lift: 32.7018494973
frozenset([44]) --> frozenset([2018]) conf: 0.502538071066  lift: 34.0053494092
frozenset([2871]) --> frozenset([5]) conf: 0.535294117647  lift: 32.7010309278
frozenset([5]) --> frozenset([2839]) conf: 0.511045655376  lift: 29.1183705838
frozenset([2871]) --> frozenset([2839]) conf: 0.505882352941  lift: 28.8241758242
frozenset([2883]) --> frozenset([5]) conf: 0.510822510823  lift: 31.2060644314
frozenset([2512]) --> frozenset([20]) conf: 0.524038461538  lift: 33.2371794872
frozenset([3018]) --> frozenset([3

In [173]:
#print confruleList

# 2d
Identify 3-4 rules and explain why you think they are interesting.

In [227]:
#this itemMeaning is going to be the dictionary from above that gives us the meaning of the values in the playlist
#for instance, the number should correspond to a song and an artist's name so we can use the rules to see who listens 
#to a given artist after listening to a specific artist

itemMeaning = index_song_dictionary

In [228]:
ap.pntRules(liftruleList,itemMeaning)  #Use this to print the rules with clear meaning

Deuces::Chris-Brown-&-Tyga
           -------->
No-Hands::Waka-Flocka-Flame
[confidence: 0.364985, lift: 28.457866]

No-Hands::Waka-Flocka-Flame
           -------->
Deuces::Chris-Brown-&-Tyga
[confidence: 0.231203, lift: 28.457866]

Colder-Weather::Zac-Brown-Band
           -------->
Who-Are-You-When-I'm-Not-Looking::Blake-Shelton
[confidence: 0.297935, lift: 37.909043]

Who-Are-You-When-I'm-Not-Looking::Blake-Shelton
           -------->
Colder-Weather::Zac-Brown-Band
[confidence: 0.309816, lift: 37.909043]

Bleed-Red::Ronnie-Dunn
           -------->
Are-You-Gonna-Kiss-Me-Or-Not?::Thompson-Square
[confidence: 0.404255, lift: 47.368674]

Are-You-Gonna-Kiss-Me-Or-Not?::Thompson-Square
           -------->
Bleed-Red::Ronnie-Dunn
[confidence: 0.268362, lift: 47.368674]

We-R-Who-We-R::Ke$ha
           -------->
What's-My-Name::Rihanna
[confidence: 0.440299, lift: 27.925966]

What's-My-Name::Rihanna
           -------->
We-R-Who-We-R::Ke$ha
[confidence: 0.180428, lift: 27.925966]

What's

# Interesting rule #1

A rule that I find interesting is:
    
We-R-Who-We-R::Ke$ha
           -------->
What's-My-Name::Rihanna
[confidence: 0.440299, lift: 27.925966]

What's-My-Name::Rihanna
           -------->
We-R-Who-We-R::Ke$ha
[confidence: 0.180428, lift: 27.925966]

In this case, the same rule is shown right next to itself, just in a different order. This tells us that playlists or user sessions that contain We-R-Who-We-R will also contain What's-My-Name with a confidence of 0.440299, but the confidence for that combination in reverse order is only 0.180428. 

The lift value is the same for both, but this difference in confidence values tells us that listeners who have Rihanna's song in their playlist won't often have Kesha's song, but user who listen to Kesha's song will also listen to Rihanna's song, with a fairly high probability of approx. 44.03%.  

# Interesting rule #2

Teenage-Dream::Katy-Perry
           -------->
DJ-Got-Us-Fallin'-In-Love::Usher
[confidence: 0.364807, lift: 33.777207]

DJ-Got-Us-Fallin'-In-Love::Usher
           -------->
Teenage-Dream::Katy-Perry
[confidence: 0.189732, lift: 33.777207]


I found this rule interesting because most of the rules that were generated have been between artists in the same genre, such as country or rap, but Katy Perry is decidedly a "pop" musician, whereas Usher is more of an "R&B/Rap" artist, so this association might be a good way for marketers to bridge the divide between pop music and rap or R&B.

This could allow someone to increase sales of R&B records by introducing young teenage fans of Katy Perry to some R&B artists they might not have heard of yet.

# Now, let's take a look at rules where we used confidence as the metric instead of life

In [229]:
ap.pntRules(confruleList,itemMeaning)  #Use this to print the rules with clear meaning

All-Of-The-Lights::Kanye-West
           -------->
Moment-4-Life::Nicki-Minaj
[confidence: 0.511727, lift: 29.157197]

Did-It-On'-Em::Nicki-Minaj
           -------->
Look-At-Me-Now::Chris-Brown
[confidence: 0.528139, lift: 49.902474]

Back-Seat::New-Boyz
           -------->
Moment-4-Life::Nicki-Minaj
[confidence: 0.617512, lift: 35.184585]

We-R-Who-We-R::Ke$ha
           -------->
Grenade::Bruno-Mars
[confidence: 0.514925, lift: 34.843564]

Look-At-Me-Now::Chris-Brown
           -------->
Moment-4-Life::Nicki-Minaj
[confidence: 0.507973, lift: 28.943278]

Look-At-Me-Now::Chris-Brown
           -------->
Down-On-Me::Jeremih
[confidence: 0.535308, lift: 32.701849]

Yeah-3X::Chris-Brown
           -------->
Grenade::Bruno-Mars
[confidence: 0.502538, lift: 34.005349]

Written-In-The-Stars::Tinie-Tempah
           -------->
Down-On-Me::Jeremih
[confidence: 0.535294, lift: 32.701031]

Down-On-Me::Jeremih
           -------->
Moment-4-Life::Nicki-Minaj
[confidence: 0.511046, lift: 29.11837

# Interesting rule #3

Another rule that I found interesting was 

I-Need_A_Doctor:: Dr.-Dre ---> Moment-4-Life::Nicki-Minaj 
                
[confidence: 0.622093, lift: 35.445630]

This is interesting to me I see Dr. Dre as being an artist who appeals to an older audience that grew up listening to public enemy in the 1980;s, whereas Nicki Minaj is a newer artist, appealing to a much younuger audience who might not have been exposed to Dr. Dre when he was in the peak of his recording career.

# Question #3
Item-Based Joke Recommendation [Dataset: jokes.zip]
For this problem you will use a modified version of the item-based recommender algorithm from Ch. 14 of Machine Learning in Action and use it on joke ratings data based on Jester Online Joke Recommender System. The modified version of the code is provided in the module itemBasedRec.py. Most of the module will be used as is, but you will add some additional functionality.

The data set contains two files. The file "modified_jester_data.csv" contains the ratings on 100 jokes by 1000 users (each row is a user profile). The ratings have been normalized to be between 1 and 21 (a 20-point scale), with 1 being the lowest rating. A zero indicates a missing rating. 

The file "jokes.csv" contains the joke ids mapped to the actual text of the jokes.

Your tasks in this problem are the following (please also see comments for the function stubs in the provided module):

In [2]:
cd /Users/zacklarsen/Desktop/CSC 478/HW/HW #4

/Users/zacklarsen/Desktop/CSC 478/HW/HW #4


In [3]:
from numpy import *
from numpy import linalg as la
import numpy as np

def euclidSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [4]:
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [5]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    data=mat(dataMat)
    U,Sigma,VT = la.svd(data)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = data.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = data[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [6]:
def cross_validate_user(dataMat, user, test_ratio, estMethod=standEst, simMeas=pearsSim):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = test_ratio * len(rated_items_by_user)
    test_indices = np.random.randint(0, len(rated_items_by_user), test_size)
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # So that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)

    # Compute absolute error for user u over all test items
    for item in withheld_items:
        # Estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])	
    
    # Now restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]
        
    # Return sum of absolute errors and the count of test cases for this user
    # Note that these will have to be accumulated for each user to compute MAE
    return error_u, count_u

In [7]:
def get_joke_text(jokes, id):
    return jokes[id]

In [114]:
# dataMat = genfromtxt('modified_jester_data.csv',delimiter=',')

# test(dataMat, 0.2, svdEst)
# test(dataMat, 0.2, standEst)

# jokes = load_jokes('jokes.csv')
# print_most_similar_jokes(dataMat, jokes, 3, 5, pearsSim)

# 3a
Load in the joke ratings data and the joke text data into appropriate data structures.

In [8]:
import StringIO
import csv

indices = []
text = []

f=open("jokes.csv")

for row in csv.reader(f):    
    joke = row[1]
    text.append(joke)

f.close()

jokes = np.array(text)

for i in range(4):
    print text[i]


A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer's disease". The man replies "Well thank God I don't have cancer!"
This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That's an awfully big word for a ten year old." 
Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.
Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.


In [9]:
ratings = genfromtxt('modified_jester_data.csv',delimiter=',')

In [10]:
ratings

array([[  3.18,  19.79,   1.34, ...,   0.  ,   0.  ,   0.  ],
       [ 15.08,  10.71,  17.36, ...,  11.34,   6.68,  12.07],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       ..., 
       [ 16.58,  16.63,  15.85, ...,   0.  ,   0.  ,   0.  ],
       [  3.67,   4.45,   3.67, ...,   3.77,   3.77,   3.28],
       [  9.88,  11.73,   9.16, ...,   0.  ,   0.  ,   0.  ]])

# 3b
Complete the definition for the function "test". This function iterates over all users and for each performs cross-validation on items (by calling the provided "cross_validate_user" function), and returns the error information necessary to compute Mean Absolute Error (MAE). 

Use this function to perform 5-fold cross-validation (i.e., 20% test-ratio) comparing MAE results using standard item-based collaborative filtering (based on the rating prediction function "standEst") with results using the SVD-based version of the rating item-based CF (using "svdEst" as the prediction engine). 

[Note: See comments provided in the module for hints on accomplishing these tasks.]

In [11]:
print ratings[1]  #Here is the ratings profile for user 1

[ 15.08  10.71  17.36  15.37   8.62   1.34  10.27   5.66  19.88  20.22
  17.75  19.64  15.42  18.43  15.56  10.03  15.66  10.32  14.3    9.79
  11.87  19.64  19.35  20.17  11.05  18.57  15.71  11.87  10.61  17.99
  17.5   10.08  18.14  20.03   9.2   11.73  18.09  14.4   10.13  18.91
  18.82   7.17  19.64  19.98   6.68   7.8    6.1   10.08   9.54  14.64
  14.16  20.03  11.97   9.69   4.5    7.8   19.64   8.86  11.1   20.03
   4.3    7.65   1.97  15.47  15.08   7.17  19.74  12.12  11.78  18.52   6.
  13.77  19.3   18.77  18.33  17.21  18.72  19.98  19.64  19.2   14.93
  15.85  15.85  17.07  19.98  15.51  10.95  14.69  15.56  11.58  13.82
   6.05  10.71  18.86  10.81   8.86  14.06  11.34   6.68  12.07]


In [12]:
def test(dataMat, test_ratio, estMethod):
    # Write this function to iterate over all users and for each perform cross-validation on items by calling
    # the above cross-validation function on each user.
    
    error_total = 0.0 #Initialize the total error and user count
    count_total = 0.0
    
    for i in range(len(dataMat)): #Here, len(dataMat) is equal to the number of users
        
        #The step below runs cross validation on user i
        error_u,count_u = cross_validate_user(dataMat, i, test_ratio, estMethod, simMeas=pearsSim) 
        
        #From the results of running cross-validation on user i, we add that user's error rating to the total
        #error rating
        error_total += error_u
        count_total += count_u
    
    
    # MAE will be the ratio of total error across all test cases to the total number of test cases, for all users
    MAE = (error_total/count_total)
    
    print 'Mean Absolute Error for ',estMethod,' : ', MAE

In [200]:
#test(ratings, 0.2, standEst)   #First run gave an MEA of 3.69400024484

In [201]:
#test(ratings, 0.2, svdEst)    #First run gave an mea of 3.61035739951

We can see from the above results that the svdEst method decreases the mean absolute error of the model by about 0.08

# 3c
Write a new function "print_most_similar_jokes" which takes the joke ratings data, a query joke id, a parameter k for the number of nearest neighbors, and a similarity metric function, and prints the text of the query joke as well as the texts of the top k most similar jokes based on user ratings. 

[Note: For hints on how to accomplish this task, please see comments at the end of the provided module as well as comments for the provided stub function.]

In [13]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [14]:
#Let's preprocess the jokes dataset
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler().fit(ratings)
ratings_norm = min_max_scaler.transform(ratings)

In [16]:
np.set_printoptions(precision=5,suppress=True)

ratings_norm

array([[ 0.15611,  0.97392,  0.06594, ...,  0.     ,  0.     ,  0.     ],
       [ 0.7403 ,  0.52707,  0.85433, ...,  0.5567 ,  0.32874,  0.59546],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       ..., 
       [ 0.81394,  0.81841,  0.78002, ...,  0.     ,  0.     ,  0.     ],
       [ 0.18017,  0.219  ,  0.18061, ...,  0.18508,  0.18553,  0.16182],
       [ 0.48503,  0.57726,  0.45079, ...,  0.     ,  0.     ,  0.     ]])

#Now that we have normalized the ratings data, we can compare a given joke to all other jokes using the rating profiles for each joke by all 1000 users. We will use different similarity measures here

In [17]:
np.set_printoptions(precision=5,suppress=True)

ratings_norm[:,4] #This is the rating for joke 4 by all 1,000 users

array([ 0.17084,  0.42317,  0.98331,  0.62838,  0.60677,  0.19195,
        0.96171,  0.13942,  0.11095,  0.85469,  0.25871,  0.57585,
        0.82818,  0.90476,  0.92145,  0.74718,  0.17771,  0.74718,
        0.66372,  0.11586,  0.66618,  0.33284,  0.37801,  0.23024,
        0.42808,  0.54737,  0.80216,  0.06087,  0.45655,  0.82818,
        0.77811,  0.70938,  0.65685,  0.44232,  0.69023,  0.55425,
        0.96662,  0.58272,  0.72361,  0.39224,  0.80707,  0.94502,
        0.85223,  0.39715,  0.82622,  0.79725,  0.62838,  0.72607,
        0.135  ,  0.10162,  0.18017,  0.5351 ,  0.60677,  0.27541,
        0.11831,  0.40403,  0.51595,  0.65439,  0.11095,  0.66863,
        0.57094,  0.3353 ,  0.93814,  0.47815,  0.21109,  0.38979,
        0.64261,  0.80707,  0.19931,  0.58517,  0.56848,  0.2327 ,
        0.20422,  0.08247,  0.6971 ,  0.54246,  0.11095,  0.838  ,
        0.94747,  0.47079,  0.65439,  0.05842,  0.87383,  0.1134 ,
        0.83554,  0.6161 ,  0.88807,  0.70692,  0.69269,  0.22

In [19]:
ratingsDF = pd.DataFrame(ratings_norm)   #Creating a pandas df just to see the data more clearly
ratingsDF.head(4)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.156112,0.973917,0.065945,0.139421,0.170839,0.122730,0.005176,0.731351,0.099165,0.307087,...,0.681796,0.000000,0.000000,0.000000,0.000000,0.000000,0.263623,0.000000,0.00000,0.000000
1,0.740304,0.527067,0.854331,0.754541,0.423171,0.065783,0.477226,0.235263,0.975945,0.995079,...,0.681796,0.298471,0.527067,0.928150,0.531988,0.437099,0.690231,0.556701,0.32874,0.595461
2,0.000000,0.000000,0.000000,0.000000,0.983309,0.995091,0.982402,0.997392,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.988189,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,0.000000,0.952264,0.000000,0.000000,0.628375,0.940599,0.369048,0.837767,0.000000,0.631890,...,0.000000,0.000000,0.000000,0.567421,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [20]:
joke4 = np.array(ratings_norm[:,3]) #Because index starts at 0, joke 4 is index 3

dists = []

for i in range(len(jokes)):
    joke = np.array(ratings_norm[:,i])
    sim = euclidSim(joke4.T,joke) 
    dist = 1-sim           #compare column i to joke4
    dists.append(dist)


In [21]:
distarray = np.array(dists)

dist_indices = distarray.argsort()

In [22]:
dist_indices

array([ 3, 23, 57, 32,  8, 56, 29, 36, 66, 42, 43,  2, 63, 58, 50, 99, 69,
       40, 59, 91, 97, 51, 98, 89,  5, 83,  0, 81, 73, 85, 94, 93, 78, 84,
       76, 86, 79, 95, 72, 90, 96, 74, 82, 54, 92, 80, 75, 22, 24, 33, 77,
       70, 87, 21, 62, 88, 16, 71, 15,  1, 39, 13, 44, 11, 10, 38,  9, 12,
       14, 19, 46, 37,  7, 17, 18, 25,  6,  4, 45, 27, 41, 47, 55, 20, 60,
       53, 30, 64, 67, 68, 65, 48, 52, 31, 26, 28, 34, 61, 35, 49])

In [23]:
for i in dist_indices:
    print jokes[i]

Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.
What do you get when you run over a parakeet with a lawnmower? Shredded tweet.
How many teddy bears does it take to change a lightbulb?It takes only one teddybear but it takes a whole lot of lightbulbs.
What do you call an American in the finals of the world cup?"Hey Beer Man!"
A country guy goes into a city bar that has a dress code and the maitred' demands he wear a tie. Discouraged the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant but says to the guy "Okay you're a pretty resourceful fellow you can come in... but just don't start anything"!  
Why are there so many Jones's in the phone book?Because they all have phones.
Q: What's the difference between a Lawyer and a Plumber? A: A Plumber work

In [24]:
jokes[3]

"Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it."

In [25]:
def print_most_similar_jokes(dataMat, jokes, queryJoke, k, metric=pearsSim):
    
    # Write this function to find the k most similar jokes (based on user ratings) to a queryJoke
    
    # The queryJoke is a joke id as given in the 'jokes.csv' file (and corresponding to the a column in dataMat)
    
    
    
    # You must compare ratings for the queryJoke (the column in dataMat corresponding to the joke), to all
    # other joke rating vectors and return the top k neighbor jokes of the queryjoke
    # Note that this is the same as performing KNN on the columns of dataMat. 
    
    
    queryJoke_rating = dataMat[:,queryJoke]
    
    dists = []

    for i in range(len(jokes)):        #For each out of 100 jokes 
        jokei = np.array(dataMat[:,i])  #Joke i is the ith column in the dataMat
        sim = metric(joke4.T,jokei)     #Compute the similarity bewtween column i and joke5
        dist = 1-sim                   #obtain the distance
        dists.append(dist)             #add distance i to the distance list
    
    
    distarray = np.array(dists)
    dist_indices = distarray.argsort()   #Here, we are sorting the distances 
    kdists_indices = dist_indices[1:k]      #and then we want to take the top k distances, ignoring the first because
                                            #the first one will be the joke compared to itself at a distance of 0
    
    
    # The function must retrieve the text of the joke from 'jokes.csv' file and print both
    # the queryJoke text as well as the text of the returned jokes.
    
    queryJoketext = jokes[queryJoke]
    
    topkjokes = []
    for i in kdists_indices:
        topkjokes.append(jokes[i])
        
    
    print queryJoketext
    for i in topkjokes:
        print i

In [26]:
# Using pearsSim as a similarity measure, and the ratings and jokes datasets, return the 5 nearest neighbors of joke
# number 3
print_most_similar_jokes(ratings_norm, jokes,3,5, euclidSim)

Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.
What do you get when you run over a parakeet with a lawnmower? Shredded tweet.
How many teddy bears does it take to change a lightbulb?It takes only one teddybear but it takes a whole lot of lightbulbs.
What do you call an American in the finals of the world cup?"Hey Beer Man!"
A country guy goes into a city bar that has a dress code and the maitred' demands he wear a tie. Discouraged the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant but says to the guy "Okay you're a pretty resourceful fellow you can come in... but just don't start anything"!  


In [27]:
print_most_similar_jokes(ratings_norm, jokes,4,10, pearsSim)

Q. What's O. J. Simpson's Internet address? A.	Slash slash backslash slash slash escape.
What do you get when you run over a parakeet with a lawnmower? Shredded tweet.
A country guy goes into a city bar that has a dress code and the maitred' demands he wear a tie. Discouraged the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant but says to the guy "Okay you're a pretty resourceful fellow you can come in... but just don't start anything"!  
Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.
What do you call an American in the finals of the world cup?"Hey Beer Man!"
Q: What's the difference between a Lawyer and a Plumber? A: A Plumber works to unclog the system.
Why are there so many Jones's in the phone book?Because they all have phones.
How many teddy bears does it take 

In [106]:
print_most_similar_jokes(ratings_norm, jokes,4,10, cosSim)   #Having problems here with cosSim

TypeError: only length-1 arrays can be converted to Python scalars

In [105]:
print_most_similar_jokes(ratings_norm, jokes,4,10, cosSim2)   #Having problems here with cosSim2

IndexError: index 652 is out of bounds for axis 1 with size 100

In [33]:
joke4

array([ 0.13942,  0.75454,  0.     ,  0.     ,  0.27541,  0.12273,
        0.     ,  0.23515,  0.20177,  0.68778,  0.     ,  0.42317,
        0.     ,  0.94747,  0.72116,  0.33726,  0.     ,  0.94502,
        0.     ,  0.     ,  0.     ,  0.     ,  0.36622,  0.12764,
        0.51841,  0.     ,  0.     ,  0.07069,  0.     ,  0.     ,
        0.68532,  0.     ,  0.68041,  0.     ,  0.82818,  0.     ,
        0.72361,  0.     ,  0.68532,  0.     ,  0.21355,  0.     ,
        0.     ,  0.66863,  0.58517,  0.26608,  0.67599,  0.51841,
        0.70447,  0.09671,  0.65439,  0.     ,  0.     ,  0.2921 ,
        0.     ,  0.     ,  0.63034,  0.     ,  0.16593,  0.     ,
        0.68532,  0.34462,  0.     ,  0.2327 ,  0.52577,  0.     ,
        0.46146,  0.70692,  0.     ,  0.62838,  0.68041,  0.     ,
        0.20619,  0.97349,  0.19686,  0.     ,  0.15611,  0.     ,
        0.4973 ,  0.     ,  0.     ,  0.     ,  0.21109,  0.15857,
        0.     ,  0.44477,  0.68041,  0.5351 ,  0.26362,  0.42

In [84]:
dataMat = ratings_norm
joke4 = np.array(dataMat[:,4])
joke1 = np.array(dataMat[:,1])

print joke1
print joke4

[ 0.97392  0.52707  0.       0.95226  0.76821  0.36713  0.       0.69685
  0.36713  0.79478  0.74213  0.62992  0.       0.99754  0.25689  0.61762
  0.08514  0.91634  0.72047  0.91634  0.50541  0.80659  0.74656  0.1688
  0.48376  0.81152  0.82825  0.54872  0.       0.74213  0.58415  0.82087
  0.69882  0.       0.46014  0.13041  0.99262  0.48622  0.68701  0.
  0.40059  0.       0.9498   0.38386  0.79232  0.95226  0.7062   0.48376
  0.5748   0.99262  0.7874   0.60335  0.       0.15453  0.83268  0.79232
  0.92618  0.53888  0.11122  0.24508  0.84498  0.59646  0.69439  0.35728
  0.07776  0.       0.45522  0.94045  0.23573  0.43406  0.84695  0.53642
  0.40994  0.99262  0.27362  0.       0.21654  0.       0.625    0.       0.
  0.84006  0.86122  0.19734  0.       0.70374  0.80906  0.06594  0.26673
  0.62254  0.48622  0.25443  0.       0.8376   0.65354  0.12549  0.29528
  0.       0.60581  0.       0.12303  0.92372  0.40059  0.40502  0.1565
  0.50541  0.40059  0.31693  0.41486  0.40502  0.58415

def euclidSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

In [100]:
def cosSim2(inA,inB):
    num = (inA.T * inB)  #TypeError: only length-1 arrays can be converted to Python scalars - this is because of the float
    #print(num)
    
    denom = la.norm(inA)*la.norm(inB)
    #print denom
    
    return 0.5 + 0.5 * (num / denom)
    
    

# This is straight from the example functions in the assignment notes

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [101]:
#When we run the code below, the cosine similarity measure returns an array instead of one similarity value

cosSim2(joke4.T,joke1)

array([ 0.50024,  0.50032,  0.5    ,  0.50086,  0.50067,  0.5001 ,
        0.5    ,  0.50014,  0.50006,  0.50097,  0.50027,  0.50052,
        0.5    ,  0.50129,  0.50034,  0.50066,  0.50002,  0.50098,
        0.50068,  0.50015,  0.50048,  0.50038,  0.5004 ,  0.50006,
        0.5003 ,  0.50064,  0.50095,  0.50005,  0.5    ,  0.50088,
        0.50065,  0.50083,  0.50066,  0.5    ,  0.50045,  0.5001 ,
        0.50137,  0.50041,  0.50071,  0.5    ,  0.50046,  0.5    ,
        0.50116,  0.50022,  0.50094,  0.50109,  0.50063,  0.5005 ,
        0.50011,  0.50014,  0.5002 ,  0.50046,  0.5    ,  0.50006,
        0.50014,  0.50046,  0.50068,  0.5005 ,  0.50002,  0.50023,
        0.50069,  0.50029,  0.50093,  0.50024,  0.50002,  0.5    ,
        0.50042,  0.50109,  0.50007,  0.50036,  0.50069,  0.50018,
        0.50012,  0.50012,  0.50027,  0.5    ,  0.50003,  0.5    ,
        0.50085,  0.5    ,  0.5    ,  0.50007,  0.50108,  0.50003,
        0.5    ,  0.50062,  0.50103,  0.50007,  0.50026,  0.50

In [103]:
pearsSim(joke4.T,joke1)

0.4764565047385238

In [104]:
euclidSim(joke4.T,joke1)

0.067908925226048175

In [ ]:
# This function is not needed for Assignment 4, but may be useful for experimentation
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1] #find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

# This function performs cross-validation on a single user based on the test_ratio
# For example, with test_ratio = 0.2, 5-fold x-validation is performed where in each fold, 
# 20 percent of rated items are withheld and the rest are used to estimate the withheld ratings